<a href="https://colab.research.google.com/github/joseluisll/AnalisisIbex/blob/main/AnalisisIbex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [354]:
# Instala plotly en Colab
!pip install plotly

import numpy as np
import pandas as pd
import plotly.graph_objects as go

**Creamos el vector de tiempos, para todos el mismo, cada posicion es un día.**

In [355]:
vector_fechas_test = pd.date_range(start='2025-01-01', end='2025-04-11')

In [370]:
vector_fechas = pd.date_range(start='2024-10-01', end='2024-12-31')

#Preparar la serie HISTORICA de IBEX35 desde 1/1/2015 A 11/04/2025

In [371]:
# Abrir la serie de IBEX35
df_ibex = pd.read_csv('datos/datos_historicos_IBEX35.csv', parse_dates=['Fecha'], index_col='Fecha')
# Convert 'Último' column to float
df_ibex['Último'] = df_ibex['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))


In [372]:
# Crear un dataframe con índice vector_fechas y valores en cero
df_ibex_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_ibex.columns)
df_ibex_reindexado = df_ibex_reindexado.astype(object)

# Copiar los valores de df_ibex a df_ibex_reindexado cuando los índices coinciden
for fecha in df_ibex.index:
  if fecha in df_ibex_reindexado.index:
    df_ibex_reindexado.loc[fecha] = df_ibex.loc[fecha]



# Method 1: Min-Max Scaling (Normalization to [0, 1])
df_ibex_reindexado['Normalizado'] = (df_ibex_reindexado['Último'] - df_ibex_reindexado['Último'].min()) / (df_ibex_reindexado['Último'].max() - df_ibex_reindexado['Último'].min())

#Preparar la serie HISTORICA de Petroleo_WTI desde 1/1/2015 A 11/04/2025

In [373]:
#datos históricos de petroleo_WTI y la normalizo
df_petroleo = pd.read_csv('datos/datos_historico_petroleo_wti.csv', parse_dates=['Fecha'], index_col='Fecha')




In [374]:
# Convert 'Último' column to float
df_petroleo['Último'] = df_petroleo['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

In [375]:
df_petroleo_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_petroleo.columns)
df_petroleo_reindexado = df_petroleo_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_petroleo.index:
  if fecha in df_petroleo_reindexado.index:
    df_petroleo_reindexado.loc[fecha] = df_petroleo.loc[fecha]


# Min-Max Scaling
df_petroleo_reindexado['Normalizado'] = (df_petroleo_reindexado['Último'] - df_petroleo_reindexado['Último'].min()) / (df_petroleo_reindexado['Último'].max() - df_petroleo_reindexado['Último'].min())


**PREPARAR LA SERIE HISTÓRICA DE VOLATILIDAD DEL VIX MISMA ESCALA TEMPORAL QUE LAS ANTERIORES**

In [376]:
# Abrir la serie de IBEX35
df_vix = pd.read_csv('datos/datos_historicos_VIX.csv', parse_dates=['Fecha'], index_col='Fecha')

# Convert 'Último' column to float
df_vix['Último'] = df_vix['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

df_vix_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_vix.columns)
df_vix_reindexado = df_vix_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_vix.index:
  if fecha in df_petroleo_reindexado.index:
    df_vix_reindexado.loc[fecha] = df_vix.loc[fecha]


# Min-Max Scaling
df_vix_reindexado['Normalizado'] = (df_vix_reindexado['Último'] - df_vix_reindexado['Último'].min()) / (df_vix_reindexado['Último'].max() - df_vix_reindexado['Último'].min())


**Metro la cotización EUR/DOL**

In [377]:
# Abrir la serie de IBEX35
df_eur_usd = pd.read_csv('datos/datos_historicos_EUR_USD.csv', parse_dates=['Fecha'], index_col='Fecha')

# Convert 'Último' column to float
df_eur_usd['Último'] = df_eur_usd['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

df_eur_usd_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_eur_usd.columns)
df_eur_usd_reindexado = df_eur_usd_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_eur_usd.index:
  if fecha in df_eur_usd_reindexado.index:
    df_eur_usd_reindexado.loc[fecha] = df_eur_usd.loc[fecha]


# Min-Max Scaling
df_eur_usd_reindexado['Normalizado'] = (df_eur_usd_reindexado['Último'] - df_eur_usd_reindexado['Último'].min()) / (df_eur_usd_reindexado['Último'].max() - df_eur_usd_reindexado['Último'].min())


#REPRESENTAR LAS SERIES TEMPORALES NORMALIZADAS SOBRE EL MISMO EJE

El grafico es interactivo y permite hacer zoom.

El OJO humano y el cerebro ya ha visto que hay relación entre ambas series.

In [378]:
# 1. Select 'Último' column from both DataFrames
df_ibex_reindexado['Derivada'] = df_ibex_reindexado['Normalizado'].diff()

ibex_ultimo = df_ibex_reindexado['Normalizado']
df_petroleo_reindexado['Derivada'] = df_petroleo_reindexado['Normalizado'].diff()

petroleo_ultimo = df_petroleo_reindexado['Normalizado']
df_vix_reindexado['Derivada'] = df_vix_reindexado['Normalizado'].diff()

vix_ultimo = df_vix_reindexado['Normalizado']

df_eur_usd_reindexado['Derivada'] = df_eur_usd_reindexado['Normalizado'].diff()
eur_usd_ultimo = df_eur_usd_reindexado['Normalizado']




# Crear DataFrame para Plotly
df_to_plot = pd.DataFrame({'IBEX35': ibex_ultimo, 'Petroleo_WTI': petroleo_ultimo, 'VIX' : vix_ultimo, 'EUR_USD' : eur_usd_ultimo},index=vector_fechas)


# Crear gráfico interactivo
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['IBEX35'],
                         mode='lines', name='IBEX35'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['Petroleo_WTI'],
                         mode='lines', name='Petroleo_WTI'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['VIX'],
                         mode='lines', name='VIX'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['EUR_USD'],
                         mode='lines', name='EUR_USD'))



umbral=0.4

df_subida = df_to_plot[(df_ibex_reindexado['Derivada'] > umbral) &
                       (df_petroleo_reindexado['Derivada'] > umbral) &
                       (df_vix_reindexado['Derivada'] > umbral) &
                       (df_eur_usd_reindexado['Derivada'] > umbral)]


# Add scatter trace for df_subida
fig.add_trace(go.Scatter(
    x=df_subida.index,
    y=df_subida['IBEX35'],  # You can choose the column to represent on y-axis
    mode='markers',
    marker=dict(color='yellow', size=8),  # Customize marker appearance
    name='Subida Points'
))

# Actualizar diseño del gráfico
fig.update_layout(title='IBEX35 vs Petroleo_WTI vs VIX vs EUR_USD',
                  xaxis_title='Fecha',
                  yaxis_title='Valor',
                  hovermode='x unified',
                  template='plotly_white')

fig.show()


Yo concluiría que el índice de petroleo_WTI puede ser un indice adelantado del IBEX 35.

¿Cómo puedo calcular el tiempo de adelanto?
¿Habrá un factor de correlación?
¿Esto se podría hacer con un modelo?.
Problemas con los ejes, tienen diferente longitud, hay que ajustar siempre al eje con la misma longitud.

In [379]:
# Assuming df_to_plot is already defined and contains your data

# Set the window size for the rolling correlation
window_size = 30  # You can adjust this to your needs

# Calculate the rolling correlation
rolling_correlation = df_to_plot['IBEX35'].rolling(window=window_size).corr(df_to_plot['Petroleo_WTI'])

# Create a new DataFrame for the rolling correlation
df_correlation = pd.DataFrame({'Correlation': rolling_correlation})
df_correlation = df_correlation.reindex(df_to_plot.index, method='pad')

# You can now plot or analyze df_correlation

In [380]:
# Add trace for the correlation
fig.add_trace(go.Scatter(x=df_correlation.index, y=df_correlation['Correlation'],
                         mode='lines', name='Correlation', yaxis='y2'))

# Update layout for the second y-axis
fig.update_layout(
    yaxis2=dict(
        title='Correlation',
        overlaying='y',
        side='right',
        range=[-1, 1]  # Set range for correlation
    ),
    title='IBEX35, Petroleo_WTI, and Correlation',
    xaxis_title='Fecha',
    yaxis_title='Valor',
    hovermode='x unified',
    template='plotly_white'
)

# Show the plot

In [381]:
df_ibex_reindexado['media_movil_3'] = df_ibex_reindexado['Último'].rolling(window=3).mean()
df_ibex_reindexado['media_movil_7'] = df_ibex_reindexado['Último'].rolling(window=7).mean()
df_ibex_reindexado['media_movil_30'] = df_ibex_reindexado['Último'].rolling(window=30).mean()

df_ibex_reindexado['Señal'] = 0  # Inicializa la señal en 0

# Define la condición para la señal de compra
condicion_compra = (df_ibex_reindexado['media_movil_3'] > df_ibex_reindexado['media_movil_7']) & \
                  (df_ibex_reindexado['media_movil_7'] > df_ibex_reindexado['media_movil_30']) & \
                  (df_ibex_reindexado['media_movil_3'].shift(1) < df_ibex_reindexado['media_movil_7'].shift(1))  # Verifica el cruce

# Asigna 1 a la señal cuando se cumple la condición
df_ibex_reindexado.loc[condicion_compra, 'Señal'] = 1

caracteristicas = ['Último', 'media_movil_3', 'media_movil_7', 'media_movil_30']
X = df_ibex_reindexado[caracteristicas]
y = df_ibex_reindexado['Señal']




ENTRENAR EL MODELO

In [389]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean') # O 'median', 'most_frequent', 'constant'
X_train = imputer.fit_transform(X_train)
print(X_train)
# Crea y entrena el modelo
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

[[11435.7         3811.9         8221.27142857  8269.50333333]
 [    0.          6991.83333333  7766.85714286  8092.11333333]
 [    0.          3843.86666667  6559.3         7480.10666667]
 [ 9984.7         6693.16666667  9504.14285714  8686.85333333]
 [    0.          3988.5         6556.75714286  8456.65239766]
 [11105.5         7293.5         8258.12597403  8456.65239766]
 [    0.          7943.23333333  8515.35714286  8456.65239766]
 [11795.3         7899.76666667  8459.54285714  8456.65239766]
 [11588.4         7754.4         8257.21428571  8793.00333333]
 [    0.          7024.96666667  7542.24285714  8690.16666667]
 [11996.7        11925.93333333  8485.71428571  8456.65239766]
 [10646.2        11047.4         9171.68571429  9050.05666667]
 [11617.9        11661.53333333  8357.35714286  8672.94666667]
 [11672.6        11727.63333333  8414.2         8167.35666667]
 [10325.7         6776.86666667  9631.07142857  8485.87666667]
 [11965.5         7874.06666667  8179.21428571  8456.65

LogisticRegression()

**EVALUAR EL MODELO**

In [388]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
# Abrir la serie de IBEX35
df_ibex_test = pd.read_csv('datos/datos_historicos_IBEX35.csv', parse_dates=['Fecha'], index_col='Fecha')
# Convert 'Último' column to float
df_ibex_test['Último'] = df_ibex_test['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
# Suponiendo que 'nuevos_datos' es un DataFrame con las nuevas observaciones
nuevos_datos_reindexado = pd.DataFrame(index=vector_fechas_test, data=0, columns=df_ibex_test.columns)

# Copiar los valores de df_ibex a df_ibex_reindexado cuando los índices coinciden
for fecha in df_ibex_test.index:
  if fecha in nuevos_datos_reindexado.index:
    nuevos_datos_reindexado.loc[fecha] = df_ibex_test.loc[fecha]

nuevos_datos_reindexado['Último'] = nuevos_datos_reindexado['Último']
nuevos_datos_reindexado['media_movil_3'] = nuevos_datos_reindexado['Último'].rolling(window=3).mean()
nuevos_datos_reindexado['media_movil_7'] = nuevos_datos_reindexado['Último'].rolling(window=7).mean()
nuevos_datos_reindexado['media_movil_30'] = nuevos_datos_reindexado['Último'].rolling(window=30).mean()

nuevos_datos_reindexado = nuevos_datos_reindexado.astype(object)

datos_entrada = nuevos_datos_reindexado[caracteristicas]
datos_entrada.fillna(0, inplace=True)
datos_entrada_imputados = imputer.fit_transform(datos_entrada)

# Realiza predicciones en el conjunto de prueba
y_pred = modelo.predict(datos_entrada_imputados)

# Filter nuevos_datos_reindexado to the dates in vector_fechas_test
filtered_data = nuevos_datos_reindexado[nuevos_datos_reindexado.index.isin(vector_fechas_test)]

# Calculate the condition for intersection of moving averages
condition = (filtered_data['media_movil_3'] > filtered_data['media_movil_7']) & \
            (filtered_data['media_movil_7'] > filtered_data['media_movil_30']) & \
            (filtered_data['media_movil_3'].shift(1) < filtered_data['media_movil_7'].shift(1))

# Create y_test based on the condition
y_test = np.where(condition, 1, 0)

print(y_pred)
print(y_test)


# Calcula las métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Accuracy: 0.8712871287128713
Precision: 0.1
Recall: 0.2


<ipython-input-388-3f179bd7296b>:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.084,00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-388-3f179bd7296b>:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.091,60' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-388-3f179bd7296b>:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.236,40' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-388-3f179bd7296b>:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '371,36M' has dtype incompatible wit